In [1]:
import numpy as np
import scanorama
import scipy.sparse as sparse
import pandas as pd

/home/dblux/anaconda3/envs/r-1/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [7]:
num_batches = 6
# odd_batches = [10,9,9,8,8,7]
# odd_cutpoints = np.cumsum(odd_batches)[:-1]

log_maqc = np.loadtxt("data/scanorama/small_log_maqc.tsv")
print(log_maqc[:5,:5])
print(log_maqc.shape)

[[10.4429197  10.0894816  10.29571872 11.23537135 11.44369902]
 [10.72762168 10.59069949 10.57948343  8.86180096  8.40812756]
 [ 7.00130038  7.06207895  6.61484286  7.57629012  7.38015883]
 [ 8.97386004  8.79325509  8.88049813  8.55848165  8.6743376 ]
 [ 0.          0.          0.          0.          0.        ]]
(54675, 36)


In [8]:
# Transpose array into samples x genes
list_arr = np.split(np.transpose(log_maqc), num_batches, axis = 0)
print(list_arr[0].shape)

file = "data/scanorama/probeset_names.txt"
with open(file, "r") as f:
    list_genes = f.readlines()

list_genes = [gene.strip() for gene in list_genes]
# Repeat list of genes for each batch
list_list_genes = [list_genes] * num_batches
print(len(list_list_genes))

(6, 54675)
6


In [9]:
# # Integration (Returns SVD embeddings)
# integrated, genes = scanorama.integrate(list_arr, list_list_genes)

# Batch correction.
list_corrected_arr, gene_colnames = scanorama.correct(list_arr, list_list_genes)

# # Integration and batch correction.
# integrated, corrected, genes = scanorama.correct(list_arr, list_list_genes, return_dimred=True)

Found 54675 genes among all datasets
[[0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]
Processing datasets (4, 5)
Processing datasets (3, 5)
Processing datasets (3, 4)
Processing datasets (2, 5)
Processing datasets (2, 4)
Processing datasets (2, 3)
Processing datasets (1, 5)
Processing datasets (1, 4)
Processing datasets (1, 3)
Processing datasets (1, 2)
Processing datasets (0, 5)
Processing datasets (0, 4)
Processing datasets (0, 3)
Processing datasets (0, 2)
Processing datasets (0, 1)


In [10]:
corrected_arr = sparse.vstack(list_corrected_arr)
corrected_arr1 = sparse.csr_matrix.transpose(corrected_arr)
print(corrected_arr1.shape)
# Replace negative values with 0
corrected_arr1[corrected_arr1 < 0] = 0
corrected_df = pd.DataFrame(corrected_arr1.toarray())
# Name rownames of df with output list from scanorama
corrected_df.index = gene_colnames
# Rows of genes not ordered the same way as input
# Sort rows according to initial rownames
corrected_df1 = corrected_df.reindex(list_genes)
print(corrected_df1.iloc[:5,:5])

(54675, 36)
                  0         1         2         3         4
1007_s_at  0.007305  0.007110  0.007218  0.007458  0.007574
1053_at    0.005852  0.005742  0.005773  0.005398  0.005219
117_at     0.004896  0.004895  0.004707  0.005048  0.004979
121_at     0.005597  0.005478  0.005539  0.005457  0.005524
1255_g_at  0.000297  0.000291  0.000297  0.000292  0.000295


In [11]:
corrected_df1.to_csv("data/scanorama/scanorama_data_small.tsv",
                     sep = "\t", header = False)